In [8]:
!kaggle datasets download -d ashaychoudhary/india-job-market-dataset --unzip -p ./india_job_market


Dataset URL: https://www.kaggle.com/datasets/ashaychoudhary/india-job-market-dataset
License(s): MIT
  0%|                                                | 0.00/406k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 406k/406k [00:00<00:00, 4.59MB/s]


In [10]:
ls

India Job Market Analysis.ipynb  india_job_market/
India-Job-Market-Analysis/       kaggle.json
README.md


In [26]:
import pandas as pd

In [27]:
df = pd.read_csv('./india_job_market/india_job_market_dataset.csv')  
df.head()

,Job ID,Job Title,Company Name,Job Location,Job Type,Salary Range,Experience Required,Posted Date,Application Deadline,Job Portal,Number of Applicants,Education Requirement,Skills Required,Remote/Onsite,Company Size
0,JOB1,Software Engineer,Amazon,Ahmedabad,Full-time,5-8 LPA,2-5 years,2025-01-16,2025-01-25,LinkedIn,23,PhD,"C++, SQL, Python",Remote,Small (1-50)
1,JOB2,Marketing Executive,Infosys,Ahmedabad,Internship,5-8 LPA,2-5 years,2024-12-25,2025-01-19,Indeed,462,MBA,"SQL, C++, Python",Remote,Large (500+)
2,JOB3,Financial Analyst,Deloitte,Jaipur,Contract,20+ LPA,5-10 years,2025-01-22,2025-01-29,Naukri.com,430,M.Tech,"Machine Learning, Excel, React",Remote,Large (500+)
3,JOB4,Business Analyst,Amazon,Delhi,Full-time,20+ LPA,2-5 years,2025-01-07,2025-02-06,LinkedIn,387,B.Tech,"Machine Learning, Python, SQL",Hybrid,Small (1-50)
4,JOB5,Software Engineer,Infosys,Delhi,Full-time,12-20 LPA,10+ years,2024-12-26,2025-01-08,Indeed,199,MBA,"UI/UX, C++, Java",Onsite,Small (1-50)


In [28]:
# City-to-coordinates mapping
city_coordinates = {
    "Ahmedabad": {"lat": 23.0225, "lon": 72.5714},
    "Jaipur": {"lat": 26.9124, "lon": 75.7873},
    "Delhi": {"lat": 28.7041, "lon": 77.1025},
    "Pune": {"lat": 18.5204, "lon": 73.8567},
    "Noida": {"lat": 28.5355, "lon": 77.3910},
    "Mumbai": {"lat": 19.0760, "lon": 72.8777},
    "Hyderabad": {"lat": 17.3850, "lon": 78.4867},
    "Kolkata": {"lat": 22.5726, "lon": 88.3639},
    "Bangalore": {"lat": 12.9716, "lon": 77.5946},
    "Chennai": {"lat": 13.0827, "lon": 80.2707}
}

In [29]:
# Map latitude and longitude
df["Latitude"] = df["Job Location"].map(lambda x: city_coordinates[x]["lat"])
df["Longitude"] = df["Job Location"].map(lambda x: city_coordinates[x]["lon"])

In [30]:
df.head()

,Job ID,Job Title,Company Name,Job Location,Job Type,Salary Range,Experience Required,Posted Date,Application Deadline,Job Portal,Number of Applicants,Education Requirement,Skills Required,Remote/Onsite,Company Size,Latitude,Longitude
0,JOB1,Software Engineer,Amazon,Ahmedabad,Full-time,5-8 LPA,2-5 years,2025-01-16,2025-01-25,LinkedIn,23,PhD,"C++, SQL, Python",Remote,Small (1-50),23.0225,72.5714
1,JOB2,Marketing Executive,Infosys,Ahmedabad,Internship,5-8 LPA,2-5 years,2024-12-25,2025-01-19,Indeed,462,MBA,"SQL, C++, Python",Remote,Large (500+),23.0225,72.5714
2,JOB3,Financial Analyst,Deloitte,Jaipur,Contract,20+ LPA,5-10 years,2025-01-22,2025-01-29,Naukri.com,430,M.Tech,"Machine Learning, Excel, React",Remote,Large (500+),26.9124,75.7873
3,JOB4,Business Analyst,Amazon,Delhi,Full-time,20+ LPA,2-5 years,2025-01-07,2025-02-06,LinkedIn,387,B.Tech,"Machine Learning, Python, SQL",Hybrid,Small (1-50),28.7041,77.1025
4,JOB5,Software Engineer,Infosys,Delhi,Full-time,12-20 LPA,10+ years,2024-12-26,2025-01-08,Indeed,199,MBA,"UI/UX, C++, Java",Onsite,Small (1-50),28.7041,77.1025


In [32]:
import plotly.express as px

# Filter by role dynamically
selected_role = "Software Engineer"
filtered_df = df[df['Job Title'] == selected_role]

# Create an interactive map
fig = px.scatter_mapbox(
    filtered_df,
    lat="Latitude",
    lon="Longitude",
    size="Number of Applicants",
    color="Job Location",
    hover_name="Job Location",
    hover_data={"Job Title": True, "Number of Applicants": True},
    size_max=50,
    zoom=4,
    title=f"Jobs for {selected_role} Across India"
)

# Mapbox style and center
fig.update_layout(mapbox_style="carto-positron", mapbox_zoom=4, mapbox_center={"lat": 20.5937, "lon": 78.9629})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()


ValueError: Value of 'hover_name' is not the name of a column in 'data_frame'. Expected one of ['Job ID', 'Job Title', 'Company Name', 'Job Location', 'Job Type', 'Salary Range', 'Experience Required', 'Posted Date', 'Application Deadline', 'Job Portal', 'Number of Applicants', 'Education Requirement', 'Skills Required', 'Remote/Onsite', 'Company Size', 'Latitude', 'Longitude'] but received: State

In [34]:
a = list(df['Number of Applicants'].unique())

In [40]:
a.sort()

In [42]:
print(a)

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227,